In [ ]:
import os
import sys
import geopandas as gpd
import pandas as pd
%matplotlib notebook
import matplotlib.pyplot as plt

In [ ]:
os.getcwd()

In [ ]:
basepath = 'C:\\Users\\Sarah\\Nexus365\\Tom Russell - ghana-oxford-infrastructure-adaptation-2020'

In [ ]:
dams_path = os.path.join(basepath, 'data/infrastructure/Points/GHA_dams.gpkg')
GHA_dams = gpd.read_file(dams_path)

#pp_fl_future_low_path = os.path.join(basepath, 'results/exposure/GHA_powerplant_new_flood_future_3-4.gpkg')
#GHA_pp_fl_fut_low = gpd.read_file(pp_fl_future_low_path)

In [ ]:
pp_fl_future_high_path = os.path.join(basepath, 'results/exposure/GHA_powerplant_OPR_new_flood_future_5-6.gpkg')
GHA_pp_fl_fut_high = gpd.read_file(pp_fl_future_high_path)

In [ ]:
#pp_ls_high_path = os.path.join(basepath, 'results/exposure/GHA_powerplant_opr__GHA_landslides_3.gpkg')
#GHA_pp_ls_high = gpd.read_file(pp_ls_high_path)

In [ ]:
admin_path = os.path.join(basepath, 'data/admin/GHA_admin1.gpkg')
GHA_admin1 = gpd.read_file(admin_path)

In [ ]:
intersections_econ_path = os.path.join(
    basepath, 'results', 'intersections_econ_merged.csv')

econ_haz_cap_path = os.path.join(
    basepath, 'results', 'econ_hazards_capacity.csv')

econ_haz_count_path = os.path.join(
    basepath, 'results','econ_hazards_sector_count_exp.csv')

econ_haz_cap_adm_path = os.path.join(
    basepath, 'results','econ_hazards_cap_admin_exp.csv')

pp_merged_admin_percent_path = os.path.join(
    basepath, 'results','pp_merged_admin_percent_exp.csv')

In [ ]:
GHA_pp

In [ ]:
GHA_pp_fl_fut_high.head(0)

In [ ]:
#GHA_pp_ls_high

In [ ]:
GHA_admin1

In [ ]:
GHA_pp_admin1 = gpd.sjoin(GHA_pp, GHA_admin1, how="inner", op='intersects')

In [ ]:
GHA_pp_admin1.head()

In [ ]:
 GHA_pp_admin1 = GHA_pp_admin1[[
        'ID',
        'NAME',
        'CAPACITY_M',
        'index_right',
        'ADMIN_1_NAME',
        'geometry']]
    
GHA_pp_admin1 = GHA_pp_admin1.rename(columns={
        'ID':'network_id','index_right': 'admin_id', 'CAPACITY_M':'CAPACITY_MW'
    })

In [ ]:
GHA_pp_admin1.head(0)

## Add hazards as columns (Intersections merged economics with each hazard as separate column)

def load_hazard(data_folder, hazard_id):  
    hazard_path = os.path.join(
        data_folder, 'data', 'hazards', '{}.shp'.format(hazard_id))
    hazards = geopandas.read_file(hazard_path)
    
hazard_ids = ['1m_sea-level','4m_storm-surge','flashflooding','landslide_susceptibility']

In [ ]:
### def intersect_hazard_with_pp(hazards, sectors, hazard_id, sector_id):
    
all_intersections = []
    
    ### INTERSECTIONS WITH High fLOOD
GHA_pp_fl_fut_high['hazard_id'] = 1

GHA_pp_fl_fut_high = GHA_pp_fl_fut_high[[
        'network_id',
         'name',
         'hazard_id',
    ]]
    
GHA_pp_fl_fut_high = GHA_pp_fl_fut_high.rename(columns={
    'hazard_id':'flood_fut_high',
})   

In [ ]:
GHA_pp_fl_fut_high.head(10)

intersections_pp_fl = intersections_pp_fl.drop_duplicates(subset='sector_id', keep='first')

In [ ]:
GHA_pp_fl_fut_high = GHA_pp_fl_fut_high.fillna(0)

all_intersections.append(GHA_pp_fl_fut_high)

    ### INTERSECTIONS WITH Low fLOOD
GHA_pp_fl_fut_low['hazard_id'] = 1

GHA_pp_fl_fut_low = GHA_pp_fl_fut_low[[
        'network_id',
         'name',
         'hazard_id',
    ]]
    
GHA_pp_fl_fut_low = GHA_pp_fl_fut_low.rename(columns={
    'hazard_id':'flood_fut_low',
})   

GHA_pp_fl_fut_low = GHA_pp_fl_fut_low.fillna(0)

all_intersections.append(GHA_pp_fl_fut_low)

In [ ]:
### INTERSECTIONS WITH LANDSLIDE

GHA_pp_ls_high['hazard_id'] = 1

GHA_pp_ls_high = GHA_pp_ls_high[[
    'network_id',
    'name',
    'hazard_id'
    ]]
    
GHA_pp_ls_high = GHA_pp_ls_high.rename(columns={
        'hazard_id': 'ls_high'
    })

GHA_pp_ls_high = GHA_pp_ls_high.fillna(0)

all_intersections.append(GHA_pp_ls_high)

In [ ]:
all_intersections = pd.concat(all_intersections, axis=0, sort=False)
all_intersections = all_intersections.fillna(0)

all_intersections.to_csv(intersections_econ_path)


In [ ]:
all_intersections.head(20)

## Hazard with cap/adm - Overview of all economic assets and whether they are exposed 

In [ ]:
merged_intersections = pd.merge(
    GHA_pp_admin1, all_intersections, how = 'outer', on=['network_id']) 
merged_intersections = merged_intersections.fillna(0)
merged_intersections.head()

In [ ]:
merged_intersections.to_csv(econ_haz_count_path, index=False)

## Add capacities to each hazard

In [ ]:
hazard_ids =['flood_fut_high']
for hazard_id in hazard_ids:
    merged_intersections[hazard_id] = merged_intersections[hazard_id]* merged_intersections['CAPACITY_MW']

merged_intersections = merged_intersections[['network_id', 'NAME', 'CAPACITY_MW', 'admin_id', 'ADMIN_1_NAME', 'flood_fut_high', 'geometry']]
merged_intersections.to_csv(econ_haz_cap_path, index=False)
merged_intersections


## Add multiple hazard calculation

In [ ]:
merged_intersections = pd.read_csv(econ_haz_cap_path)

In [ ]:
list =['flood_fut_high']
#ensure all values are numeric
merged_intersections[list] = merged_intersections[list].apply(pd.to_numeric)

**!!!**

**doesn't it make more sense not to include capacity in the list, so that this is not sumed up, but then join it afterwards?**

**!!!**

In [ ]:
# Merge information for each asset - grouping by 
# -> If network_id is the same, add the columns for each of the hazards
merged_intersections = merged_intersections.groupby([
    'network_id', 'ADMIN_1_NAME', 'geometry'
    ])[
    list
    ].sum()
merged_intersections

In [ ]:
merged_intersections = merged_intersections.merge(GHA_pp_admin1, on='network_id')
merged_intersections

## Calculate per Admin district

In [ ]:
list =['flood_fut_high','CAPACITY_MW']

In [ ]:
# Trying to merge information for each Admin district - grouping by 
# -> If ADMIN_1_NAME is the same, add the columns for each of the hazards
merged_districts = merged_intersections.groupby([
    'ADMIN_1_NAME',
    ])[
    list
    ].sum()
merged_districts

In [ ]:
merged_districts.to_csv(econ_haz_cap_adm_path)

In [ ]:
merged_distr_percent = pd.read_csv(econ_haz_cap_adm_path)

hazard_ids =['flood_fut_high']
for hazard_id in hazard_ids:
    merged_distr_percent[hazard_id] = merged_distr_percent[hazard_id]/ merged_distr_percent['CAPACITY_MW']*100
    merged_distr_percent = merged_distr_percent.rename(columns = {hazard_id:hazard_id+'_%'})
    
merged_distr_percent = merged_distr_percent.drop('CAPACITY_MW', 1)
merged_distr_percent = merged_districts.merge(merged_distr_percent, on='ADMIN_1_NAME')

merged_distr_percent = merged_distr_percent.merge(GHA_admin1, on='ADMIN_1_NAME')


merged_distr_percent

In [ ]:
#merged_distr_percent_all = pd.concat(GHA_admin1, merged_distr_percent, sort=False)

In [ ]:
merged_distr_percent.to_csv(pp_merged_admin_percent_path)